# 🎯 Визуализация Агента Интервью

Этот ноутбук демонстрирует работу агента интервью с использованием LangGraph и OpenRouter.ai

## Структура агента:
- **Нода 1**: Получение имени кандидата
- **Нода 2**: Получение возраста кандидата
- **Нода 3**: Генерация персональных вопросов


In [ ]:
# Импорты
import sys
import os
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch, ConnectionPatch
import networkx as nx
from IPython.display import display, HTML, JSON
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src
sys.path.append('../src')

print("📦 Библиотеки загружены успешно!")


: 

In [ ]:
# Импортируем наш агент
from interview_agent import InterviewAgent, InterviewState
from openrouter_client import OpenRouterClient
from config import config

print("🤖 Агент интервью загружен!")


## 1. Визуализация Workflow с NetworkX


In [ ]:
def create_workflow_graph():
    """Создать граф workflow агента"""
    
    # Создаем граф
    G = nx.DiGraph()
    
    # Добавляем ноды
    nodes = {
        'start': {'label': '🚀 START', 'color': '#4CAF50', 'description': 'Начало интервью'},
        'get_name': {'label': '👤 ИМЯ', 'color': '#2196F3', 'description': 'Получение имени кандидата'},
        'get_age': {'label': '🎂 ВОЗРАСТ', 'color': '#FF9800', 'description': 'Получение возраста кандидата'},
        'generate_questions': {'label': '❓ ВОПРОСЫ', 'color': '#9C27B0', 'description': 'Генерация персональных вопросов'},
        'end': {'label': '✅ END', 'color': '#F44336', 'description': 'Завершение интервью'}
    }
    
    # Добавляем ноды в граф
    for node_id, node_data in nodes.items():
        G.add_node(node_id, **node_data)
    
    # Добавляем связи
    edges = [
        ('start', 'get_name'),
        ('get_name', 'get_age'),
        ('get_age', 'generate_questions'),
        ('generate_questions', 'end')
    ]
    
    G.add_edges_from(edges)
    
    return G, nodes

# Создаем граф
G, node_info = create_workflow_graph()
print(f"📊 Граф создан: {G.number_of_nodes()} нод, {G.number_of_edges()} связей")


In [ ]:
def visualize_workflow():
    """Визуализировать workflow агента"""
    
    plt.figure(figsize=(14, 10))
    
    # Позиционирование нод
    pos = {
        'start': (1, 4),
        'get_name': (1, 3),
        'get_age': (1, 2),
        'generate_questions': (1, 1),
        'end': (1, 0)
    }
    
    # Рисуем ноды
    for node_id, (x, y) in pos.items():
        node_data = node_info[node_id]
        
        # Создаем красивый прямоугольник
        bbox = FancyBboxPatch(
            (x-0.4, y-0.2), 0.8, 0.4,
            boxstyle="round,pad=0.05",
            facecolor=node_data['color'],
            edgecolor='black',
            linewidth=2,
            alpha=0.8
        )
        plt.gca().add_patch(bbox)
        
        # Добавляем текст
        plt.text(x, y, node_data['label'], 
                ha='center', va='center', 
                fontsize=12, fontweight='bold',
                color='white')
        
        # Добавляем описание
        plt.text(x, y-0.35, node_data['description'], 
                ha='center', va='center', 
                fontsize=9, style='italic',
                color='gray')
    
    # Рисуем стрелки
    for edge in G.edges():
        start_pos = pos[edge[0]]
        end_pos = pos[edge[1]]
        
        # Создаем стрелку
        arrow = patches.FancyArrowPatch(
            (start_pos[0], start_pos[1] - 0.2),
            (end_pos[0], end_pos[1] + 0.2),
            arrowstyle='->',
            mutation_scale=20,
            color='black',
            linewidth=2
        )
        plt.gca().add_patch(arrow)
    
    # Настройки графика
    plt.xlim(0, 2)
    plt.ylim(-0.5, 4.5)
    plt.axis('off')
    plt.title('🎯 Workflow Агента Интервью', fontsize=16, fontweight='bold', pad=20)
    
    # Добавляем легенду
    legend_elements = [
        patches.Patch(color='#4CAF50', label='Начало'),
        patches.Patch(color='#2196F3', label='Сбор данных'),
        patches.Patch(color='#FF9800', label='Обработка'),
        patches.Patch(color='#9C27B0', label='Генерация'),
        patches.Patch(color='#F44336', label='Завершение')
    ]
    plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1))
    
    plt.tight_layout()
    plt.show()

# Визуализируем workflow
visualize_workflow()


## 2. Демонстрация работы агента


In [ ]:
# Создаем экземпляр агента
agent = InterviewAgent()

print("🤖 Агент создан!")
print(f"📊 Конфигурация OpenRouter: {'✅ Настроена' if config.is_configured() else '❌ Не настроена'}")
if config.is_configured():
    print(f"🧠 Модель: {config.model}")
else:
    print("🔄 Будет использоваться fallback режим")


In [ ]:
# Симулируем работу агента
def simulate_agent_workflow():
    """Симулировать работу агента без интерактивного ввода"""
    
    print("🎬 СИМУЛЯЦИЯ РАБОТЫ АГЕНТА")
    print("=" * 50)
    
    # Создаем тестовые данные
    test_state = {
        "messages": [],
        "candidate_name": "Анна Петрова",
        "candidate_age": 28,
        "candidate_position": "Python разработчик",
        "interview_questions": [],
        "current_step": "start",
        "ai_responses": [],
        "analysis_results": []
    }
    
    # Симулируем прохождение через ноды
    print("\n1️⃣ НОДА: Получение имени")
    print(f"   📝 Имя кандидата: {test_state['candidate_name']}")
    test_state["messages"].append({"role": "user", "content": test_state['candidate_name']})
    test_state["current_step"] = "name_collected"
    
    print("\n2️⃣ НОДА: Получение возраста")
    print(f"   📝 Возраст кандидата: {test_state['candidate_age']} лет")
    test_state["messages"].append({"role": "user", "content": str(test_state['candidate_age'])})
    test_state["current_step"] = "age_collected"
    
    print("\n3️⃣ НОДА: Генерация вопросов")
    # Генерируем вопросы
    questions = agent.openrouter_client.generate_interview_questions(
        test_state['candidate_name'],
        test_state['candidate_age'],
        test_state['candidate_position']
    )
    test_state["interview_questions"] = questions
    test_state["current_step"] = "questions_generated"
    
    print(f"   📝 Сгенерировано {len(questions)} вопросов")
    
    return test_state

# Запускаем симуляцию
simulated_state = simulate_agent_workflow()


In [ ]:
# Показываем результаты симуляции
print("\n📋 РЕЗУЛЬТАТЫ СИМУЛЯЦИИ")
print("=" * 50)

print(f"👤 Кандидат: {simulated_state['candidate_name']}")
print(f"🎂 Возраст: {simulated_state['candidate_age']} лет")
print(f"💼 Позиция: {simulated_state['candidate_position']}")
print(f"📝 Вопросов: {len(simulated_state['interview_questions'])}")
print(f"🔄 Текущий шаг: {simulated_state['current_step']}")

print("\n📋 СГЕНЕРИРОВАННЫЕ ВОПРОСЫ:")
for i, question in enumerate(simulated_state['interview_questions'][:5], 1):
    print(f"{i}. {question}")

if len(simulated_state['interview_questions']) > 5:
    print(f"... и еще {len(simulated_state['interview_questions']) - 5} вопросов")


## 3. Визуализация состояния агента


In [ ]:
def visualize_agent_state(state):
    """Визуализировать состояние агента"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Информация о кандидате
    candidate_info = [
        state.get('candidate_name', 'Не указано'),
        f"{state.get('candidate_age', 0)} лет",
        state.get('candidate_position', 'Не указано')
    ]
    
    ax1.barh(['Имя', 'Возраст', 'Позиция'], [1, 1, 1], 
             color=['#2196F3', '#FF9800', '#9C27B0'])
    ax1.set_title('👤 Информация о кандидате', fontweight='bold')
    ax1.set_xlim(0, 1.2)
    
    # Добавляем текст
    for i, info in enumerate(candidate_info):
        ax1.text(0.5, i, info, ha='center', va='center', 
                fontweight='bold', color='white')
    
    # 2. Прогресс интервью
    steps = ['Имя', 'Возраст', 'Вопросы', 'Завершение']
    progress = [1, 1, 1, 1]  # Все шаги выполнены в симуляции
    
    ax2.bar(steps, progress, color=['#4CAF50', '#4CAF50', '#4CAF50', '#4CAF50'])
    ax2.set_title('📊 Прогресс интервью', fontweight='bold')
    ax2.set_ylabel('Статус')
    
    # 3. Количество вопросов
    question_count = len(state.get('interview_questions', []))
    ax3.pie([question_count, max(0, 10-question_count)], 
            labels=['Сгенерировано', 'Осталось'],
            colors=['#9C27B0', '#E0E0E0'],
            autopct='%1.0f%%')
    ax3.set_title('❓ Вопросы интервью', fontweight='bold')
    
    # 4. Статистика сообщений
    messages = state.get('messages', [])
    user_messages = len([m for m in messages if m.get('role') == 'user'])
    assistant_messages = len([m for m in messages if m.get('role') == 'assistant'])
    
    ax4.bar(['Пользователь', 'Агент'], [user_messages, assistant_messages],
            color=['#2196F3', '#FF9800'])
    ax4.set_title('💬 Сообщения в диалоге', fontweight='bold')
    ax4.set_ylabel('Количество')
    
    plt.tight_layout()
    plt.show()

# Визуализируем состояние
visualize_agent_state(simulated_state)


## 4. JSON представление состояния


In [ ]:
# Показываем JSON представление состояния
print("📄 JSON ПРЕДСТАВЛЕНИЕ СОСТОЯНИЯ АГЕНТА:")
print("=" * 50)

# Создаем красивое JSON представление
json_state = {
    "candidate_info": {
        "name": simulated_state.get('candidate_name', ''),
        "age": simulated_state.get('candidate_age', 0),
        "position": simulated_state.get('candidate_position', '')
    },
    "interview_data": {
        "total_questions": len(simulated_state.get('interview_questions', [])),
        "current_step": simulated_state.get('current_step', ''),
        "messages_count": len(simulated_state.get('messages', []))
    },
    "questions": simulated_state.get('interview_questions', [])[:3],  # Первые 3 вопроса
    "workflow_status": "completed"
}

# Отображаем JSON
display(JSON(json_state, expanded=True))


## 5. Заключение

Этот ноутбук демонстрирует:

1. **Структуру workflow** агента интервью
2. **Визуализацию нод** и их связей
3. **Симуляцию работы** агента
4. **Анализ состояния** агента
5. **JSON представление** данных

Агент успешно использует LangGraph для создания структурированного процесса интервью с интеграцией OpenRouter.ai для генерации персональных вопросов.

### Основные возможности:
- ✅ **Две основные ноды**: получение имени и возраста
- ✅ **Интеграция с OpenRouter.ai**: использование языковых моделей
- ✅ **Fallback режим**: работа без API ключа
- ✅ **Визуализация**: наглядное представление workflow
- ✅ **Симуляция**: тестирование без интерактивного ввода
